<a href="https://colab.research.google.com/github/Jushef/Azure-AutoML/blob/main/AutoML%20Jupyter%20Notebook/Iris_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cargar librerías Azure
from azureml.core import Workspace, Dataset, Datastore, Experiment
from azureml.core.compute import ComputeTarget
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails

# Cargar bibliotecas que no son de Azure
import pandas as pd
import numpy as np

In [ ]:
# Compruebe la versión base de AzureMl
import azureml.core
print("You are currently using version " + azureml.core.VERSION + " of the Azure ML SDK")

You are currently using version 1.37.0 of the Azure ML SDK


In [ ]:
# Recupere su espacio de trabajo 
ws = Workspace.from_config()

In [ ]:
# Recuperar el objetivo informático para ejecutar AutoML de forma remota
compute_name = 'automl-compute'
compute_target = ComputeTarget(ws, compute_name)

In [ ]:
# Recupere su Datastore por nombre rellenando los valores en minúsculas entre comillas dobles
datastore = Datastore.get_default(ws)
my_datastore_name = 'workspaceblobstore'
my_datastore = Datastore.get(ws, my_datastore_name)

In [ ]:
# Si ya ha registrado su conjunto de datos, recupérelo
dataset_name = "Iris"
dataset = Dataset.get_by_name(ws, dataset_name, version='latest')

In [ ]:
# Vea su conjunto de datos convirtiéndolo en pandas
dataset.take(10).to_pandas_dataframe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
5,6,5.4,3.9,1.7,0.4,Iris-setosa
6,7,4.6,3.4,1.4,0.3,Iris-setosa
7,8,5.0,3.4,1.5,0.2,Iris-setosa
8,9,4.4,2.9,1.4,0.2,Iris-setosa
9,10,4.9,3.1,1.5,0.1,Iris-setosa


In [ ]:
dataset.to_pandas_dataframe().Species.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [ ]:
# Cree un nombre para su experimento AutoML de clasificación del Titanic
experiment_name = 'Iris-Multi-Classification'

# Establezca el experimento en su espacio de trabajo
exp = Experiment(workspace=ws, name=experiment_name)

In [ ]:
# Establezca las variables para su ejecución de AutoML
target_column = 'Species'
task = 'classification'
primary_metric = 'accuracy'
featurization = 'auto'

In [ ]:
# Establezca el número de clases únicas en su columna de destino
num_classes = 3

In [ ]:
# Establezca su configuración AutoML
config = AutoMLConfig(task=task,
                     primary_metric=primary_metric,
                     num_classes=num_classes,
                     featurization=featurization,
                     compute_target=compute_target,
                     training_data=dataset,
                     label_column_name=target_column,
                     experiment_timeout_minutes=15,
                     enable_early_stopping=True,
                     n_cross_validations=5,
                     model_explainability=True,
                     enable_stack_ensemble=True,
                     enable_voting_ensemble=True)

In [ ]:
# Entrene su modelo de clasificación AutoML Titanic
AutoML_run = exp.submit(config, show_output = True)
RunDetails(AutoML_run).show()

Submitting remote run.
No run_configuration provided, running on automl-compute with default configuration
Running on remote compute: automl-compute


Experiment,Id,Type,Status,Details Page,Docs Page
Iris-Multi-Classification,AutoML_bbf31a09-9397-4435-ab0e-38e924b59cde,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [ ]:
# Registre su modelo AutoML entrenado en su espacio de trabajo AMLS
description = 'AutoML Classification Run using Iris Data.' 
tags = {'project' : "Iris", "creator" : "your name"} 
model_name = 'Iris-Classification-AutoML' 

AutoML_run.register_model(model_name=model_name,description=description,tags=tags)

Model(workspace=Workspace.create(name='automl-example-workspace', subscription_id='0aad4149-65b5-42c8-9b25-ce17f2fed264', resource_group='auto-ml-example-resource-group'), name=Iris-Classification-AutoML, id=Iris-Classification-AutoML:1, version=1, tags={'project': 'Iris', 'creator': 'your name'}, properties={})